In [1]:
# -*- coding: utf8 -*-
import numpy as np
import collections
import tensorflow as tf
import json

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
with open('./data/modified/int_word.json', 'r') as f:
    int_word = json.load(f)
with open('./data/modified/word_int.json', 'r') as f:
    word_int = json.load(f)
with open('./data/modified/trans_poems.json', 'r') as f:
    trans_poems = np.asarray(json.load(f))
vocab_size = len(int_word)
sequence_len = np.average([len(p) for p in trans_poems])

In [94]:
batch_size = 256
epochs = 20
batch_num = len(trans_poems) // batch_size
capacity=50000
min_after_dequeue=10000
print(batch_num)

167


In [95]:
tf.reset_default_graph()
trans_poems = np.asarray(trans_poems).astype(np.int32)
poem_queue = tf.train.slice_input_producer([trans_poems], shuffle=False)
get_batch = tf.train.shuffle_batch(poem_queue, batch_size, capacity, min_after_dequeue, num_threads=4, seed=345)
getX = tf.slice(get_batch, [0, 0] ,[batch_size, int(sequence_len - 1)])
getY = tf.slice(get_batch, [0, 1] ,[batch_size, int(sequence_len - 1)])

In [96]:
embed_size = 128
rnn_cell_num = 2
rnn_cell_size = 100
learning_rate = 0.001

In [97]:
def build_model(getX):
    with tf.variable_scope('embedding', reuse=tf.AUTO_REUSE):
        embedding_matrix = tf.get_variable('embedding_matrix', shape=[vocab_size, embed_size],
                                           initializer=tf.random_uniform_initializer(-1, 1, seed=123))
        embedding = tf.nn.embedding_lookup(embedding_matrix, getX)
        tf.summary.histogram('embed', embedding_matrix)
    with tf.variable_scope('rnn', reuse=tf.AUTO_REUSE):
        lstms = tf.contrib.rnn.MultiRNNCell(
            [tf.contrib.rnn.LSTMCell(rnn_cell_size, tf.random_uniform_initializer(-1, 1, seed=456)) 
             for _ in range(rnn_cell_num)])
        init_states = lstms.zero_state(np.shape(getX)[0], dtype=tf.float32)
        outputs, states = tf.nn.dynamic_rnn(lstms, embedding, dtype=tf.float32, initial_state=init_states)
        tf.summary.histogram('rnn_cell_state', states)
    with tf.variable_scope('loss', reuse=tf.AUTO_REUSE):
        w = tf.get_variable('w', shape=[rnn_cell_size, vocab_size], 
                           initializer=tf.random_uniform_initializer(-1, 1, seed=234))
        b = tf.get_variable('b', shape=[vocab_size], 
                           initializer=tf.zeros_initializer())
        tf.summary.histogram('w', w)
        tf.summary.histogram('b', b)
        _outputs = tf.reshape(outputs, [-1, rnn_cell_size])
        logits = tf.matmul(_outputs, w) + b
        return logits, init_states, states

In [98]:
logits, _, _ = build_model(getX)
with tf.variable_scope('loss', reuse=tf.AUTO_REUSE):
    _getY = tf.reshape(getY, [-1, 1])
    loss = tf.reduce_mean(
        tf.contrib.legacy_seq2seq.sequence_loss_by_example(
            [logits], [_getY], [tf.ones_like([_getY], dtype=tf.float32)], vocab_size))
    tf.summary.scalar('loss', loss)
    
with tf.variable_scope('train_op', reuse=tf.AUTO_REUSE):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), 5)
    global_step = tf.Variable(0, trainable=False, name='global_step', dtype=tf.int32)
    train_op = optimizer.apply_gradients(zip(grads, tvars), global_step=global_step)
    
merged_summary_op = tf.summary.merge_all()

In [99]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    summary_writer = tf.summary.FileWriter('./logs/', sess.graph)
    saver = tf.train.Saver()
    checkpoint = tf.train.get_checkpoint_state('./checkpoints/')
    if checkpoint and checkpoint.model_checkpoint_path:
        saver.restore(sess, checkpoint.model_checkpoint_path)
        print('Restored: ', checkpoint.model_checkpoint_path)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess, coord)
    try:
        while not coord.should_stop():
            for e in range(epochs):
                for i in range(batch_num):
                    _, cost, summary, g_step = sess.run([train_op, loss, merged_summary_op, global_step])
                    print('Epoch {} Batch {} Iter {} Loss {}'.format(e, i, e*batch_num + i, cost))
                    summary_writer.add_summary(summary, g_step)
                    if g_step % 10 == 0:
                        saver.save(sess, './checkpoints/poem', global_step=g_step)
    except tf.errors.OutOfRangeError:
        print('done!')
    finally:
        coord.request_stop()
        coord.join(threads)
    summary_writer.close()

INFO:tensorflow:Restoring parameters from ./checkpoints/poem-1750
Restored:  ./checkpoints/poem-1750
Epoch 0 Batch 0 Iter 0 Loss 1.72725510597229
Epoch 0 Batch 1 Iter 1 Loss 1.6497809886932373
Epoch 0 Batch 2 Iter 2 Loss 1.6591544151306152
Epoch 0 Batch 3 Iter 3 Loss 1.6023144721984863
Epoch 0 Batch 4 Iter 4 Loss 1.6554136276245117
Epoch 0 Batch 5 Iter 5 Loss 1.6578608751296997
Epoch 0 Batch 6 Iter 6 Loss 1.60295832157135
Epoch 0 Batch 7 Iter 7 Loss 1.6686408519744873
Epoch 0 Batch 8 Iter 8 Loss 1.5949310064315796
Epoch 0 Batch 9 Iter 9 Loss 1.6345000267028809
Epoch 0 Batch 10 Iter 10 Loss 1.6360524892807007
Epoch 0 Batch 11 Iter 11 Loss 1.6488592624664307
Epoch 0 Batch 12 Iter 12 Loss 1.5931370258331299
Epoch 0 Batch 13 Iter 13 Loss 1.6855632066726685
Epoch 0 Batch 14 Iter 14 Loss 1.6566179990768433
Epoch 0 Batch 15 Iter 15 Loss 1.6231762170791626
Epoch 0 Batch 16 Iter 16 Loss 1.6247591972351074
Epoch 0 Batch 17 Iter 17 Loss 1.6196705102920532
Epoch 0 Batch 18 Iter 18 Loss 1.562110424

KeyboardInterrupt: 

In [78]:
tf.reset_default_graph()
test_X = tf.placeholder(tf.int32, shape=[1, 1], name='X_for_test')
test_logits, test_init_state, test_state = build_model(test_X)
test_prob = tf.nn.softmax(test_logits)

In [79]:
def get_word(probs):
    a = np.cumsum(probs)
    s = np.sum(probs)
    b = np.random.rand(1)
    idx = int(np.searchsorted(a, b))
    return idx, int_word.get(str(idx))

In [82]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    checkpoint = tf.train.get_checkpoint_state('./checkpoints/')
    if checkpoint and checkpoint.model_checkpoint_path:
        saver.restore(sess, checkpoint.model_checkpoint_path)
    else:
        print('No checkpoint found!')
    idx = 0 # 以 [ 开头
    p, t_state = sess.run([test_prob, test_state], {test_X: [[idx]]})
    idx, word = get_word(p)
    poem = word
    while word != ']' and word != '_': #
        p, t_state = sess.run([test_prob, test_state], {test_X: [[idx]], test_init_state: t_state})
        idx, word = get_word(p)
        poem += word
    print(poem[:-1])      

INFO:tensorflow:Restoring parameters from ./checkpoints/poem-1750
常听王家镇，崇才助兄词。海仇横建代，弦甲九成成。雨雁归云里，舟冲送客泉。希方论病素，永哲盛闲论。
